# 3D model to study the heat transfer along the fracture

This script corresponds to the model used in section 3.3.5 (Heat transfer along fracture) of my master's thesis report.

In [1]:
!jupyter --version

Selected Jupyter core packages...
IPython          : 8.8.0
ipykernel        : 6.20.2
ipywidgets       : 7.7.2
jupyter_client   : 7.4.9
jupyter_core     : 5.1.4
jupyter_server   : 2.1.0
jupyterlab       : 3.5.3
nbclient         : 0.7.2
nbconvert        : 7.2.8
nbformat         : 5.7.3
notebook         : 6.5.2
qtconsole        : 5.4.0
traitlets        : 5.8.1


In [2]:
import numpy as np
import io
from mpi4py import MPI
import pyvista
import ufl
from ufl import Measure, FacetNormal
import matplotlib.pyplot as plt

import dolfinx
from dolfinx import fem, plot
from dolfinx.io import XDMFFile
from dolfinx.fem import FunctionSpace, VectorFunctionSpace, Constant, Function
from dolfinx.plot import create_vtk_mesh
from dolfinx.io.gmshio import read_from_msh 

from petsc4py import PETSc
from petsc4py.PETSc import ScalarType

## Mesh reading

In [3]:
mesh, cell_tags, facet_tags = read_from_msh("../../meshes/3D/3D_fracture_mesh.msh", MPI.COMM_WORLD, 0)

Info    : Reading '../../Meshes/3D/3D_fracture_mesh.msh'...
Info    : 396 entities
Info    : 326800 nodes
Info    : 317300 elements                                               
Info    : Done reading '../../Meshes/3D/3D_fracture_mesh.msh'              


In [4]:
print(np.unique(cell_tags.values))
print(np.unique(facet_tags.values))

[103 104 105 107]
[1 2]


Uncomment the following cell to visualize the mesh and the different subdomains.

## Output file

In [5]:
xdmf = XDMFFile(mesh.comm, "solution_3D_fracture_model.xdmf", "w")
xdmf.write_mesh(mesh)

## Temporal parameters

In [6]:
t = 0              # start time
T = 8*3600         # final time
num_steps = 200    # 200
dt = T / num_steps # time step size

Text = 20 # initial temperature in the system
Tinj = 70 # temperature of injected water

## Finite element function space for temperature field

In [7]:
V = FunctionSpace(mesh, ("CG", 1))  # Lagrange linear elements (degree 1)

## Initial conditions

In [8]:
T_n = Function(V)
T_n.name = "T_n"
T_n.x.array[:] = np.full(len(T_n.x.array), Text)

Store the initial condition in another variable as T_n undergoes changes at each iteration, ensuring that the initial temperature is preserved for calculating different energies at each step.

In [9]:
T_i = T_n.copy()
T_i.name = "T_i"

## Time-dependent output

In [10]:
T_h = T_n.copy()
T_h.name = "T_h"
xdmf.write_function(T_h, t)

## Trial and test functions

In [11]:
T, r = ufl.TrialFunction(V), ufl.TestFunction(V)

## Material properties

In [12]:
# for gabbro
rho_g = 3000                 # density of gabbro in kg/m³
c_g = 460                    # specific heat of gabbro in J/(kg*K)
cond_g = 2.15                # thermal conductivity of gabbro in W/(m·K)
print("Thermal diffusivity of gabbro:" + str(cond_g/(rho_g*c_g)))

# for water
rho_w = 997                  # density of water in kg/m³
c_w = 4182                   # specific heat of water in J/(kg*K)
cond_w = 0.598               # thermal conductivity of water in W/(m·K)
print("Thermal diffusivity of water:" + str(cond_w/(rho_w*c_w)))

# for metal (assume steel)
rho_m = 8000                 # density of metal in kg/m³
c_m = 420                    # specific heat of metal in J/(kg*K)
cond_m = 45                  # thermal conductivity of metal in W/(m·K)
print("Thermal diffusivity of metal:" + str(cond_m/(rho_m*c_m)))

# for epoxy resin
rho_e = 1100                 # density of epoxy resin in kg/m³
c_e = 1110                   # specific heat of epoxy resin in J/(kg*K)
cond_e = 0.14                # thermal conductivity of epoxy resin in W/(m·K)
print("Thermal diffusivity of epoxy resin:" + str(cond_e/(rho_e*c_e)))

# give those different properties to the domain
M = FunctionSpace(mesh, ("DG", 0))
metal_mask = (cell_tags.values == 103)
epoxy_mask = (cell_tags.values == 104)
rock_mask = (cell_tags.values == 105)
fluid_mask = (cell_tags.values == 106)|(cell_tags.values == 107)

rho = Function(M)
rho.x.array[metal_mask] = np.full(metal_mask.sum(), rho_g)
rho.x.array[epoxy_mask] = np.full(epoxy_mask.sum(), rho_g)
rho.x.array[rock_mask] = np.full(rock_mask.sum(), rho_g)
rho.x.array[fluid_mask] = np.full(fluid_mask.sum(), rho_w)

c = Function(M)
c.x.array[metal_mask] = np.full(metal_mask.sum(), c_g)
c.x.array[epoxy_mask] = np.full(epoxy_mask.sum(), c_g)
c.x.array[rock_mask] = np.full(rock_mask.sum(), c_g)
c.x.array[fluid_mask] = np.full(fluid_mask.sum(), c_w)

cond = Function(M)
cond.x.array[metal_mask] = np.full(metal_mask.sum(), cond_g)
cond.x.array[epoxy_mask] = np.full(epoxy_mask.sum(), cond_g)
cond.x.array[rock_mask] = np.full(rock_mask.sum(), cond_g)
cond.x.array[fluid_mask] = np.full(fluid_mask.sum(), cond_w)

Thermal diffusivity of gabbro:1.5579710144927536e-06
Thermal diffusivity of water:1.4342405504413766e-07
Thermal diffusivity of metal:1.3392857142857142e-05
Thermal diffusivity of epoxy resin:1.1466011466011467e-07


## Fluid flow

In [13]:
# 0.25mL/min flow rate is injected by the pump
Qinj = 0.25e-6/60*8*20 # *8*20 to test advection dominated regime
Dpipe = 0.001          # inner diameter of the pipe is 1mm
e = 0.001/2            # assumed fracture width (500 microns)

# Finite element function space for fluid velocity field
Q = VectorFunctionSpace(mesh, ("DG", 0))
q = Function(Q)
num_cells = mesh.topology.index_map(mesh.topology.dim).size_local
block_size = Q.dofmap.index_map_bs  # number of dof per dofmap
# define radial flux in the fracture and 0 flux in the solid materials (rock, tubing, epoxy)
for i in range(num_cells):
    # in rock the flow is null
    if (cell_tags.values[i] == 103) | (cell_tags.values[i] == 104) | (cell_tags.values[i] == 105) :
        q.x.array[[i*block_size, i*block_size+1, i*block_size+2]] = [0., 0., 0.]
        # in the fracture the flow is radial and depends on the distance r to the well 
    elif cell_tags.values[i] == 107:
        # obtain coordinates of the centroid of the element
        coord_centroid = Q.tabulate_dof_coordinates()[i]
        # calculate radial distance between the well center and the centroid of the element
        rad = np.linalg.norm(coord_centroid[:2])
        # calculate unit vector in the direction of flux
        u = coord_centroid[:2]/rad
        # calculate the flow rate norm as a function of the distance from the well and the pump flow
        norm = Qinj/(2*np.pi*rad*e)
        # assign the flow
        q.x.array[[i*block_size, i*block_size+1, i*block_size+2]] = list(norm*u) + [0.]

In [14]:
print(4*Qinj/(np.pi*Dpipe**2))
print(Qinj/(2*np.pi*Dpipe/2*e))

0.8488263631567753
0.4244131815783876


Uncomment the following cell to visualize the fluid velocity field inside the domain.

## Boundary conditions

In [15]:
# DIRICHLET: T=Ti on side 1
boundary_dofs_inj = fem.locate_dofs_topological(V, mesh.topology.dim-1, facet_tags.indices[facet_tags.values == 1])
bc_inj = fem.dirichletbc(ScalarType(Tinj), boundary_dofs_inj, V)

bc_tot = [bc_inj]

## Variational problem

In [16]:
F = rho * c * T * r * ufl.dx + dt * cond * ufl.dot(ufl.grad(T), ufl.grad(r)) * ufl.dx + dt * rho_w * c_w * ufl.dot(q, ufl.grad(T)) * r * ufl.dx \
    - (rho * c * T_n * r * ufl.dx)

In [17]:
a = ufl.lhs(F)
L = ufl.rhs(F)

## Linear algebra structures for the time dependent problem

In [18]:
bilinear_form = fem.form(a)
linear_form = fem.form(L)

In [19]:
%%time
# bilinear_form (a) isn't time dependant so we can assemble it once
A = fem.petsc.assemble_matrix(bilinear_form, bcs=bc_tot)
A.assemble()
b = fem.petsc.create_vector(linear_form)

CPU times: user 1.67 s, sys: 44 ms, total: 1.72 s
Wall time: 1.71 s


## Custom integration measures

In [20]:
# integrate over subdomains
dx = Measure("dx", domain=mesh, subdomain_data=cell_tags)
# integrate over boundaries
ds = Measure("ds", domain=mesh, subdomain_data=facet_tags)
n = FacetNormal(mesh)

## Linear solver

In [21]:
%%time
solver = PETSc.KSP().create(mesh.comm)
solver.setOperators(A)
solver.setType(PETSc.KSP.Type.PREONLY)
solver.getPC().setType(PETSc.PC.Type.LU)

CPU times: user 305 µs, sys: 1 µs, total: 306 µs
Wall time: 2.33 ms


In [22]:
%%time
for i in range(num_steps):
    t += dt

    # Update the right hand side reusing the initial vector
    with b.localForm() as loc_b:
        loc_b.set(0)
    fem.petsc.assemble_vector(b, linear_form)
    
    # Apply Dirichlet boundary condition to the vector
    fem.petsc.apply_lifting(b, [bilinear_form], [bc_tot])
    b.ghostUpdate(addv=PETSc.InsertMode.ADD_VALUES, mode=PETSc.ScatterMode.REVERSE)
    fem.petsc.set_bc(b, bc_tot)

    # Solve linear problem
    solver.solve(b, T_h.vector)
    T_h.x.scatter_forward()

    # Update solution at previous time step (u_n)
    T_n.x.array[:] = T_h.x.array
    
    # Write solution to file
    xdmf.write_function(T_h, t)
        
    if i % 10 == 0:
        print(i)
        
xdmf.close()

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
CPU times: user 47min 25s, sys: 18min 12s, total: 1h 5min 37s
Wall time: 1h 5min 27s
